# Navigation Control A - Process Data

Note: this includes the logic for updating the database for excluded participants

In [1]:
import urllib.request
from functools import lru_cache
import datetime
import csv, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from msdm.domains import GridWorld
from msdm.algorithms import PolicyIteration
from vgc_project import gridutils, sampsat

In [2]:
# load experiment parameters
expConfig = json.load(open("config.json", 'r'))
EXPERIMENT_CODE_VERSION = expConfig['params']['EXPERIMENT_CODE_VERSION']
cond1 = expConfig['timelines'][0]
basegrids = json.load(open("../mazes/mazes_0-11.json", "r"))

CREDENTIALS = json.load(open("../credentials.json", 'r'))
EXPURL = CREDENTIALS["EXPURL"]
USERNAME = CREDENTIALS["USERNAME"]
PASSWORD = CREDENTIALS["PASSWORD"]
print("Credentials:")
print(CREDENTIALS)

DESTDIR = "./data/"
if not os.path.exists(DESTDIR):
    os.mkdir(DESTDIR)
sourcedest = [
    (f"data/{EXPERIMENT_CODE_VERSION}/trialdata", DESTDIR+"rawtrialdata.csv"),
    (f"data/{EXPERIMENT_CODE_VERSION}/questiondata", DESTDIR+"rawquestiondata.csv"),
    (f"data/{EXPERIMENT_CODE_VERSION}/bonusdata", DESTDIR+"rawbonusdata.csv"),
    (f"data/{EXPERIMENT_CODE_VERSION}/conditiondata", DESTDIR+"rawconditiondata.csv")
]

password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_mgr.add_password(None, EXPURL, USERNAME, PASSWORD)
handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

In [3]:
for SOURCE, DEST in sourcedest:
    opener = urllib.request.build_opener(handler)
    opener.open(EXPURL+SOURCE)
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(EXPURL+SOURCE, DEST)   

In [4]:
csv.field_size_limit(sys.maxsize)
trialdata = [line for line in csv.reader(open(sourcedest[0][1], 'r'))]
questiondata = [line for line in csv.reader(open(sourcedest[1][1], 'r'))]
bonusdata = [line for line in csv.reader(open(sourcedest[2][1], 'r'))]
conditiondata = [line for line in csv.reader(open(sourcedest[3][1], 'r'))]

In [5]:
psiturk_conditions = pd.DataFrame(conditiondata, columns=['psiturk_id', 'condition'])

In [6]:
trials = []
errors = []
for psiturk_id, idx, datetime_ms, entry in trialdata:
    if "debug" in psiturk_id:
        continue
    datetime_ms = datetime.datetime.fromtimestamp(float(datetime_ms)/1000.)
    idx = int(idx)
    entry = json.loads(entry)
    try:
        assert idx == entry['trial_index']
    except AssertionError as e :
        errors.append((e, psiturk_id, idx, datetime_ms, entry))
    trials.append({
        "psiturk_id": psiturk_id,
        "idx": idx,
        "datetime_ms": datetime_ms,
        "data": entry.get("data", None),
        "trial_type": entry['trial_type'],
        "time_elapsed": entry['time_elapsed'],
        "internal_node_id": float(entry["internal_node_id"].split("-")[1])
    })
trials = pd.DataFrame(trials)
trials = trials[trials.psiturk_id.apply(lambda p: "debug" not in p)]

In [7]:
print(trials.groupby('psiturk_id')['internal_node_id'].max().value_counts())

104.0    178
1.0        5
5.0        3
21.0       1
0.0        1
52.0       1
7.0        1
2.0        1
Name: internal_node_id, dtype: int64


In [8]:
def process_navigationData(row):
    nd = row['navigationData']
    new_nd = []
    try:
        g = basegrids[row['grid']]
    except KeyError:
        g = basegrids[f"grid-{row['grid'].split('-')[1]}-0"] #older coding of grid
        
    trans = row['transform']
    for t in nd:
        nt = {**t}
        nt['trans_state'] = t['state']
        nt['trans_nextstate'] = t['nextstate']
        s = gridutils.untransformState[trans](g, t['state'])
        nt['state'] = (int(round(s[0])), int(round(s[1])))
        ns = gridutils.untransformState[trans](g, t['nextstate'])
        nt['nextstate'] = (int(round(ns[0])), int(round(ns[1])))
        nt['rt'] = t['response_datetime'] - t['start_datetime']
        new_nd.append(nt)
    return new_nd

In [9]:
#breadcrumb trials
bc_trials = trials[(trials.trial_type == "GridNavBreadcrumbs")]
bc_trials = bc_trials[(bc_trials.data.apply(lambda d: d['trialparams']['roundtype'] != "practice"))]
bc_trials = pd.concat([
    pd.DataFrame({
        "psiturk_id": bc_trials.psiturk_id,
    }).reset_index(drop=True),
    pd.DataFrame(list(bc_trials.data.apply(lambda d: {**d['trialparams'], **{k: v for k, v in d.items() if k not in ['trialparams', 'taskparams']}}).values))
], axis=1)
bc_trials = bc_trials[['original_gridname', 'original_sessionId', 'psiturk_id', 'sessionId', 'grid', 
                       'transform', 'round', "navigationData", "allCollected"]].reset_index(drop=True)
bc_trials['navigationData'] = bc_trials.apply(process_navigationData, axis=1)
bc_trials['initial_rt'] = bc_trials['navigationData'].apply(lambda traj: traj[0]['rt'])
bc_trials['max_noninitial_rt'] = bc_trials['navigationData'].apply(lambda traj: max([t['rt'] for t in traj[1:]]))
bc_trials['totaltime'] = bc_trials['navigationData'].apply(lambda traj: max([t['response_datetime'] for t in traj]) - min([t['start_datetime'] for t in traj]))

In [10]:
#awareness trials
attn_trials = trials[(trials.trial_type == "GridBlockAttentionQuery")]
attn_trials = pd.concat([
    pd.DataFrame({
        "psiturk_id": attn_trials.psiturk_id,
    }).reset_index(drop=True),
    pd.DataFrame(list(attn_trials.data.values))
], axis=1)
attn_trials['attn_resp'] = attn_trials['response'].apply(int)
attn_trials['attn_resp_N'] = (attn_trials['attn_resp'] - 1)/(8 - 1)
attn_trials['rt'] = attn_trials['responsetime'] - attn_trials['starttime']
attn_trials['obstacle'] = attn_trials['probeobs']
attn_trials['transform'] = attn_trials['trans']
attn_trials = attn_trials[['psiturk_id', 'sessionId', 'grid', 'transform', 'round', 'starttime', 'responsetime',
                           'queryround', 'obstacle', 'rt', 'attn_resp', 'attn_resp_N']].reset_index(drop=True)

In [11]:
#survey level data
survey_trials = trials[(trials.trial_type.isin(["CustomSurvey", "SaveGlobalStore"]))]
survey_trials = pd.concat([
    pd.DataFrame({
        "psiturk_id": survey_trials.psiturk_id,
    }).reset_index(drop=True),
    pd.DataFrame([d if d else {} for d in survey_trials.data.values])
], axis=1)
survey_trials = survey_trials.melt(id_vars='psiturk_id', var_name="question", value_name="response")
survey_trials = survey_trials[~survey_trials.response.isna()]
survey_trials = survey_trials.pivot(index='psiturk_id', columns="question" )
survey_trials.columns = survey_trials.columns.droplevel()
survey_trials = survey_trials.reset_index()
survey_trials.columns.name = None
survey_trials = survey_trials[survey_trials['playedGame'].isin(["Yes", "No"])].reset_index(drop=True)
completed_task = survey_trials['psiturk_id']

In [15]:
# calculate exclusions
survey_trials['exclude_participant'] = survey_trials.apply(
    lambda r: "yes" in r['playedGame'].lower(),
    axis=1
)
bc_trials["exclude_trial"] = bc_trials.apply(
    lambda r: (r['initial_rt'] > 5000) or (r['max_noninitial_rt'] > 2000) or (not r['allCollected']),
    axis=1
)
attn_trials = attn_trials.merge(bc_trials[["sessionId", "round", "exclude_trial"]], on=['sessionId', 'round'])
exclude_df = bc_trials.groupby(['psiturk_id', 'sessionId'])['exclude_trial'].apply(lambda et: (((12 - len(et)) + sum(et))/12 > .2)).reset_index(name="exclude_trials")\
    .merge(survey_trials[['sessionId', 'psiturk_id', 'exclude_participant', 'condition']], on=['psiturk_id','sessionId'], how='outer')
exclude_df['any_exclude'] = exclude_df['exclude_trials'] | exclude_df['exclude_participant']
sid_to_exclude = set(exclude_df[exclude_df['any_exclude']]['sessionId'].values)
pid_to_exclude = set(exclude_df[exclude_df['any_exclude']]['psiturk_id'].values)
survey_trials['exclude'] = survey_trials.sessionId.isin(sid_to_exclude)
del survey_trials['exclude_participant']
assert len(sid_to_exclude) == len(pid_to_exclude)
all_pid = set([pid for pid, cond in conditiondata])
pid_no_data = all_pid - (set(survey_trials.psiturk_id) | set(bc_trials.psiturk_id))
print(f"Total Participants: {len(survey_trials)}")
print(f"Participants with too many invalid trials: {exclude_df['exclude_trials'].sum()}")
print(f"Participants who previously played task: {exclude_df['exclude_participant'].sum()}")
print(f"Participants to Exclude: {len(sid_to_exclude)}")
print(f"Participants to Exclude from survey_trials: {survey_trials.exclude.sum()}")
print(f"Final non-excluded participants: {(~survey_trials.exclude).sum()}")
print(f"Participants with no data: {len(pid_no_data)}")

Total Participants: 178
Participants with too many invalid trials: 13
Participants who previously played task: 6
Participants to Exclude: 18
Participants to Exclude from survey_trials: 15
Final non-excluded participants: 163
Participants with no data: 26


In [13]:
# updates psiturk database to recruit conditions appropriately
# it is idempodent
for i, pid in enumerate(pid_to_exclude | pid_no_data):
    new_status = "ignore"
    print(i, pid)
    request = f"http://frozen-depths-23358.herokuapp.com/set_status?uniqueId={pid}&new_status={new_status}"
    res = urllib.request.urlopen(request).read()
    res = json.loads(res)
    if "success" not in res['status'].lower():
        print(res['status'], request)

0 60c89aa864cf115a31804ab4:60ca764380e1ab7e05a7af0e
1 5e94f09f191821358403de33:60ca37e1a85d23004a2d31ee
2 5fa9392c0cc79c03b3f4eb58:60ca5a240adc683e53743e65
3 60c1042feef36a30f677c2ca:60ca599a7b6b5bdfd0d0556c
4 5f20879043c28117afb27c08:60ca40647a11d53d9a5ffc05
5 60ca12851efed59c2389f3b4:60ca41a6078e5d20fb527ec9
6 5870336cdb3e720001e230cd:60ca74170667fa6e983dc617
7 5d5d65f84bd7800017a4f90e:60ca58cf7aee6134385e7fae
8 60ca047934d623e05913741c:60ca3b488fff23fdcc54c7bd
9 5994d7d9c825cf0001d7dd30:60ca73c75f390927ead37d67
10 5e8b861386bb470531a727d4:60ca73caf3e0cca5cadf5cf7
11 5bf04db81e78580001dc266a:60ca73fc0cab9d6195ed8d9d
12 606a033071a5927f9fe0ebd1:60ca3fb3981214c31bb247b0
13 5f2928bbeeb55d03021cbdc5:60ca360f9c1b3fcfb01ca828
14 5b6f312393405d000159e471:60caaaaac1703f16f1bd8f28
15 5d56add07d795e001788bb04:60cb45622aa27b456192f097
16 5b28df1a2bc7780001c2322f:60ca5a3f45f45a7eb45ec69c
17 598caf6833481c000164d13d:60ca7420f027b2f29d13ce7a
18 5cac639638d61000014da640:60ca6ed18a94fef405ecd752
19 

In [14]:
# save data before exclusions
survey_trials.to_json("./data/all-survey-trials.json")
attn_trials.to_json("./data/all-attention-trials.json")
bc_trials.to_json("./data/all-nav-trials.json")

# exclusions
survey_trials = survey_trials[~survey_trials.exclude].reset_index(drop=True)
attn_trials = attn_trials[~attn_trials['exclude_trial'] & attn_trials.sessionId.isin(survey_trials.sessionId)].reset_index(drop=True)
bc_trials = bc_trials[~bc_trials['exclude_trial'] & bc_trials.sessionId.isin(survey_trials.sessionId)].reset_index(drop=True)

# data after exclusions
# survey_trials.to_json("./data/survey-trials.json")
# attn_trials.to_json("./data/attention-trials.json")
# bc_trials.to_json("./data/nav-trials.json")

In [15]:
# final conditions counts
survey_trials.condition.value_counts().value_counts()

1    157
2      3
Name: condition, dtype: int64

In [16]:
set(range(160)) - set(survey_trials.condition)

set()

In [17]:
# sanity check that bc trials and attention trial counts line up
assert all(bc_trials.groupby('sessionId')['round'].count() > (12*.8))
assert len(bc_trials) == len(set(zip(attn_trials.sessionId, attn_trials['round'])))

# Bonusing

In [18]:
bonusdf = pd.DataFrame(bonusdata, columns=['psiturk_id', 'bonus'])
bonusdf['bonus'] = bonusdf['bonus'].apply(float)
bonusdf = bonusdf[bonusdf['psiturk_id'].isin(completed_task)]
tot_b = 0
for _, r in bonusdf.iterrows():
    b = max(r.bonus - .2, 0)
    if b == 0.0:
        continue
    tot_b += b
    print(f"{r.psiturk_id.split(':')[0]},{b:.2f}")
print(tot_b)

5f0c72c24903aa5e4bfa5495,1.20
5a465b1756ec1100014a317f,1.20
59f1de0177860e00015138fe,1.20
5936d77cebf54a0001566901,1.20
60c3ffe6d7f1a8c25d490fab,0.70
5f8bb4f68cb05a2f5c15130a,1.20
5d837955f1038a001f8d4ed6,1.10
5fbfaf3b2c47c32f27b34f28,1.20
5ea6db215a4011076de48877,1.20
5e2db13181b0be3a4e0156a9,1.20
5afbf02302b1b5000103e313,1.20
60c86ccaa8e47fbf909b736f,1.20
561021e87ffc8a0005811358,1.20
5f2aa04952f67f2f20d13b0e,1.20
56cf3dce31a5bc000de1bec8,1.20
5d56add07d795e001788bb04,0.10
5a6ad584d5d4cb0001d63d52,1.20
566d5997d408ea00051a2a86,1.20
60be2463a18facb6178dfd71,1.20
5e8e35ae9950ef0c39fb9e79,1.20
60c987af70262cd98758ae81,1.20
60ca047934d623e05913741c,1.20
60ca044c7b2e9a4654efeb77,1.20
60bcbe0d2c872dac0bb1396e,1.20
5e25d748cf445f81e521f802,1.00
5e9d06b70f475923732775c1,1.20
5bf3feb0345b9b00016ead01,1.20
59e113cdae9f950001d83f0c,1.20
59f110d1be399f0001e4a69f,1.00
601d69a993d94008fb2b25dc,1.20
603a5abe4e3f3262290f7b87,1.10
60c8f44c9d44e6308ddbc50f,1.20
5ffc6c7890ccb7070c8cfd22,1.20
60c9ea33ab